# Opt algo visualizer

This notebook reads the pickle produced by `tests.py` and visualizes it

In [52]:
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [53]:
out = pickle.load(open('tests.pickle', 'rb'))
out.keys()
out[('DataModel', 'RandomSearch')][2]

,z,func,history,x
0,196,1.073332,"[[1, 3.5191454614463664], [2, 3.14618202653340...",2.1878838345966054|-0.6602439969338354|2.78977...
1,196,1.051083,"[[1, 2.0636080824631002], [2, 1.94219051565865...",0.09432258090428292|1.2625557426287948|-2.5959...
2,196,1.122425,"[[1, 3.1481951070507486], [2, 2.69300737228224...",0.5672079886709782|-2.625842459512885|0.052340...
3,196,1.219109,"[[1, 3.6226742289913076], [2, 3.26291487058997...",-1.1027834425790108|1.7704087226388534|-2.3698...
4,196,1.248949,"[[1, 3.706273126207154], [2, 3.359338367070498...",-1.2553671443729686|1.9701326519739801|-0.4149...


In [54]:
benchmark_to_plot = 'DataModel'
algo_to_plot      = 'SMAC'

The code below will plot the trace for this algorithm and this benchmark. Click on the pan button, and then
  - Left-click and drag to pan
  - Right-click and drag to zoom

In [55]:
%matplotlib qt

df       = out[(benchmark_to_plot, algo_to_plot)][2].copy()
best_obj = out[(benchmark_to_plot, algo_to_plot)][1]

df_exploded        = df.explode('history').reset_index(drop=True)
df_exploded['obj'] = df_exploded.history.str[1].astype(float)

fig, ax = plt.subplots(figsize=(12, 8))

df_exploded.obj.cummin().plot(linewidth=0, marker='o', markersize=3, fillstyle='none', color='red')

for this_group, this_df in df_exploded.groupby('x'):
    plt.plot(this_df.index, this_df.obj, marker='x', markersize=3, linestyle='-', linewidth=0.3, color='black')

plt.yscale('log')

plt.title(f'Connected lines correspond to a evaluations\nfor a single value of x at different fidelities\n(Minimum thus far in red; final minimum: {round(best_obj, 3)})')
plt.xlabel('Function evaluation number')
plt.ylabel('Function value (log scale)')

sns.despine()

In [56]:
import numpy as np
x,y,history,_ = out[('DataModel', 'SMAC')]
x

print(x)
proportions = np.exp(x) / np.sum(np.exp(x))
print(proportions)
print(y)

[ 2.82216163 -2.53117548 -2.69713016 -2.79824854 -2.51834313]
[0.98313179 0.00465251 0.00394107 0.00356204 0.00471259]
0.8599456658412621


In [57]:
import numpy as np
x,y,history,_ = out[('DataModel', 'RandomSearch')]
x

print(x)
proportions = np.exp(x) / np.sum(np.exp(x))
print(proportions)
print(y)

[ 0.09432258  1.26255574 -2.59598988  0.08861135  2.98182494]
[0.04305354 0.13847331 0.0029216  0.04280835 0.7727432 ]
1.0510827719468696
